In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver = webdriver.Firefox()

In [2]:
driver.get("https://www.gpeters.com/names/baby-names.php")
assert "Baby Name Guesser" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

/tmp/ipykernel_9329/4043983876.py:3: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  elem = driver.find_element_by_name("q")


In [5]:
driver.close()